In [ ]:
import requests
import pandas as pd
from concurrent.futures import as_completed, ProcessPoolExecutor
from requests_futures.sessions import FuturesSession
from requests.adapters import HTTPAdapter
import time
from urllib3.util.retry import Retry
from configparser import ConfigParser
parser = ConfigParser()
parser.read('config.ini')
import json
from datetime import datetime
import traceback
from supabase import create_client, Client

# from supabase import create_client
url = str(parser.get('supabase', 'SUPABASE_URL'))
key= str(parser.get('supabase', 'SUPABASE_KEY'))
api_key = str(parser.get('riot_api', 'api_key'))
api_key = "api_key="+api_key
supabase: Client = create_client(url, key)

def api_get_match_history_ids(puuid=None, region='europe', start=0, count=10):

    """Gets the match history ids for a given puuid.

    Args:
        puuid (str, optional): Player's puuid. Defaults to None.
        region (str, optional): Player's region. Defaults to 'americas'.
        start (int, optional): Match # start (for pagination). Defaults to 0.
        count (int, optional): How many matches per page. Defaults to 100.

    Returns:
        list: List of match ids.
    """

    try:
        root_url = f'https://{region}.api.riotgames.com'
        endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
        query_params = f'?&start={start}&count={count}'

        
        while True:
            response = requests.get(root_url+endpoint+query_params+'&'+api_key)
            if response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', 10))  # Default to 10 seconds
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            elif response.status_code == 200:  # Request successful
                return response.json()
            else:
                print(f"Unexpected status code: {response.status_code}. Retrying...")
                time.sleep(5)  # Wait for 5 seconds before retrying
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    
def process_match_json_reporting(match_json):
    """Processes the match json into a dataframe.

    Args:
        match_json (dict): Match JSON.
        puuid (str): Player's puuid.

    Returns:
        dataframe: Dataframe of the processed match data.
    """
    side_dict = {
        100:'blue',
        200:'red'
    }
    info = match_json['info']
    metadata = match_json['metadata']
    matchId = metadata['matchId']
    participants = metadata['participants']
    
    matchDFs = pd.DataFrame()
    
    for player_puuid in participants:
             
        player = info['participants'][participants.index(player_puuid)]

        #get opponent champion 
        for other_player in info['participants']:
            if other_player['teamPosition'] == player['teamPosition'] and other_player['championName'] != player['championName']:
                opp_champion = other_player['championName']
            else:
                opp_champion = None 

        gameCreation = info['gameCreation']
        gameStartTimestamp = info['gameStartTimestamp']
        gameEndTimestamp = info['gameEndTimestamp']
        timePlayed = gameEndTimestamp-gameStartTimestamp
        gameMode = info['gameMode']
        gameVersion = info['gameVersion']
        platformId = info['platformId']
        queueId = info['queueId']
        puuid = player['puuid']
        riotIdGameName = player['summonerName']
        try:
            riotIdTagLine = player['riotIdTagline']
        except:
            riotIdTagLine = ''
        side = side_dict[player['teamId']]
        win = player['win']

        champion = player['championName']
        kills = player['kills']
        deaths = player['deaths']
        assists = player['assists']
        summOne = player['summoner1Id']
        summTwo = player['summoner2Id']
        earlySurrender = player['gameEndedInEarlySurrender']
        surrender = player['gameEndedInSurrender']
        firstBlood = player['firstBloodKill']
        firstBloodAssist = player['firstBloodAssist']
        firstTower = player['firstTowerKill']
        firstTowerAssist = player['firstTowerAssist']
        dragonKills = player['dragonKills']

        damageDealtToBuildings = player['damageDealtToBuildings']
        damageDealtToObjectives = player['damageDealtToObjectives']
        damageSelfMitigated = player['damageSelfMitigated']
        goldEarned = player['goldEarned']
        teamPosition = player['teamPosition']
        lane = player['lane']
        largestKillingSpree = player['largestKillingSpree']
        longestTimeSpentLiving = player['longestTimeSpentLiving']
        objectivesStolen = player['objectivesStolen']
        totalMinionsKilled = player['totalMinionsKilled']
        totalAllyJungleMinionsKilled = player['totalAllyJungleMinionsKilled']
        totalEnemyJungleMinionsKilled = player['totalEnemyJungleMinionsKilled']
        totalNeutralMinionsKilled = totalAllyJungleMinionsKilled + totalEnemyJungleMinionsKilled
        totalDamageDealtToChampions = player['totalDamageDealtToChampions']
        totalDamageShieldedOnTeammates = player['totalDamageShieldedOnTeammates']
        totalHealsOnTeammates = player['totalHealsOnTeammates']
        totalDamageTaken = player['totalDamageTaken']
        totalTimeCCDealt = player['totalTimeCCDealt']
        totalTimeSpentDead = player['totalTimeSpentDead']
        turretKills = player['turretKills']
        turretsLost = player['turretsLost']
        visionScore = player['visionScore']
        controlWardsPlaced = player['detectorWardsPlaced']
        wardsKilled = player['wardsKilled']
        wardsPlaced = player['wardsPlaced']

        item0 = player['item0']
        item1 = player['item1']
        item2 = player['item2']
        item3 = player['item3']
        item4 = player['item4']
        item5 = player['item5']
        item6 = player['item6']
        try:
            perks = player['perks']

            perkKeystone = perks['styles'][0]['selections'][0]['perk']
            perkPrimaryRow1 = perks['styles'][0]['selections'][1]['perk']
            perkPrimaryRow2 = perks['styles'][0]['selections'][2]['perk']
            perkPrimaryRow3 = perks['styles'][0]['selections'][3]['perk']
            perkPrimaryStyle = perks['styles'][0]['style']
            perkSecondaryRow1 = perks['styles'][1]['selections'][0]['perk']
            perkSecondaryRow2 = perks['styles'][1]['selections'][1]['perk']
            perkSecondaryStyle = perks['styles'][1]['style']
            perkShardDefense = perks['statPerks']['defense']
            perkShardFlex = perks['statPerks']['flex']
            perkShardOffense = perks['statPerks']['offense']
        except:
            perkKeystone = ''
            perkPrimaryRow1 = ''
            perkPrimaryRow2 = ''
            perkPrimaryRow3 = ''
            perkPrimaryStyle = ''
            perkSecondaryRow1 = ''
            perkSecondaryRow2 = ''
            perkSecondaryStyle = ''
            perkShardDefense = ''
            perkShardFlex = ''
            perkShardOffense = ''


        matchDF = pd.DataFrame({
            'match_id': [matchId],
            'participants': [participants],
            'game_creation': [gameCreation],
            'game_start_timestamp': [gameStartTimestamp],
            'game_end_timestamp': [gameEndTimestamp],
            'game_version': [gameVersion],
            'queue_id': [queueId],
            'game_mode': [gameMode],
            'platform_id': [platformId],
            'puuid': [puuid],
            'riot_id': [riotIdGameName],
            'riot_tag': [riotIdTagLine],
            'time_played': [timePlayed],
            'side': [side],
            'win': [win],
            'team_position': [teamPosition],
            'lane': [lane],
            'champion': [champion],
            'kills': [kills],
            'deaths': [deaths],
            'assists': [assists],
            'summoner1_id': [summOne],
            'summoner2_id': [summTwo],
            'gold_earned': [goldEarned],
            'total_minions_killed': [totalMinionsKilled],
            'total_neutral_minions_killed': [totalNeutralMinionsKilled],
            'total_ally_jungle_minions_killed': [totalAllyJungleMinionsKilled],
            'total_enemy_jungle_minions_killed': [totalEnemyJungleMinionsKilled],
            'early_surrender': [earlySurrender],
            'surrender': [surrender],
            'first_blood': [firstBlood],
            'first_blood_assist': [firstBloodAssist],
            'first_tower': [firstTower],
            'first_tower_assist': [firstTowerAssist],
            'damage_dealt_to_buildings': [damageDealtToBuildings],
            'turret_kills': [turretKills],
            'turrets_lost': [turretsLost],
            'damage_dealt_to_objectives': [damageDealtToObjectives],
            'dragonKills': [dragonKills],
            'objectives_stolen': [objectivesStolen],
            'longest_time_spent_living': [longestTimeSpentLiving],
            'largest_killing_spree': [largestKillingSpree],
            'total_damage_dealt_champions': [totalDamageDealtToChampions],
            'total_damage_taken': [totalDamageTaken],
            'total_damage_self_mitigated': [damageSelfMitigated],
            'total_damage_shielded_teammates': [totalDamageShieldedOnTeammates],
            'total_heals_teammates': [totalHealsOnTeammates],
            'total_time_crowd_controlled': [totalTimeCCDealt],
            'total_time_spent_dead': [totalTimeSpentDead],
            'vision_score': [visionScore],
            'wards_killed': [wardsKilled],
            'wards_placed': [wardsPlaced],
            'control_wards_placed': [controlWardsPlaced],
            'item0': [item0],
            'item1': [item1],
            'item2': [item2],
            'item3': [item3],
            'item4': [item4],
            'item5': [item5],
            'item6': [item6],
            'perk_keystone': [perkKeystone],
            'perk_primary_row_1': [perkPrimaryRow1],
            'perk_primary_row_2': [perkPrimaryRow2],
            'perk_primary_row_3': [perkPrimaryRow3],
            'perk_secondary_row_1': [perkSecondaryRow1],
            'perk_secondary_row_2': [perkSecondaryRow2],
            'perk_primary_style': [perkPrimaryStyle],
            'perk_secondary_style': [perkSecondaryStyle],
            'perk_shard_defense': [perkShardDefense],
            'perk_shard_flex': [perkShardFlex],
            'perk_shard_offense': [perkShardOffense],
            'opp_champion': [opp_champion],

        })
        matchDFs = pd.concat([matchDFs,matchDF]) 
    return matchDFs

def api_get_match_history_puuid(list_puuid, region='europe', debug=False, reporting_focus = False):
    
    """Gets the match history for a given riot_id and riot_tag.

    Args:
        list_puuid(list): Player's puuid. One for each team.
        region (str, optional): Player's region. Defaults to 'americas'.
        debug (bool, optional): Whether or not to print out matchIds as they are processed. Defaults to False.
        reporting_focus (bool, optional): Whether or not to focus on only picks and winrate. Defaults to False.

    Returns:
        DataFrame: DataFrame of all matches.
    """

    # Set up a session with retry mechanisms
    session = FuturesSession(executor=ProcessPoolExecutor(max_workers=10))
    retries = 5
    status_forcelist = [429]
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        respect_retry_after_header=True,
        status_forcelist=status_forcelist,
    )

    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    list_matchIds = []
    for player_puuid in list_puuid:
        print(player_puuid)
        matchIds = api_get_match_history_ids(puuid=player_puuid)
        for id in matchIds: #append the value to the list_matchIds and not lists
            list_matchIds.append(id)

    print('Nombre de matchs avec doublons:',len(list_matchIds))
    print('Nombre de matchs sans doublons:',len(list(set(list_matchIds))))

    df = pd.DataFrame()
    list_matchIds = list(set(list_matchIds))

    if len(list_matchIds) > 0:

        # If there are new matches to process, create asynchronous requests for match data
        futures = [session.get(f'https://{region}.api.riotgames.com/lol/match/v5/matches/{matchId}' + '?' + api_key) for matchId in list_matchIds]

        i = 0

        # Iterate through completed asynchronous requests
        for future in as_completed(futures):
            resp = future.result()
            try:
                x = resp.json()['metadata']['matchId']
            except : 
                    print('Answer :', resp.json())
                
            if debug:
                # If debug is enabled, print match processing information
                t1 = time.time()
                df = pd.concat([df, process_match_json_reporting(resp.json())])

                t2 = time.time()
                
                print('a',resp.json()['metadata']['matchId'] + f' - {i} ({round(t2 - t1, 2)}s)')
         
                i += 1
            else:
                df = pd.concat([df, process_match_json_reporting(resp.json())])
                    

        try:# Return the DataFrame containing information about the fetched matches
            df['game_creation'] = pd.to_datetime(df['game_creation'], unit='ms')
            df['game_start_timestamp'] = pd.to_datetime(df['game_start_timestamp'], unit='ms')
            df['game_end_timestamp'] = pd.to_datetime(df['game_end_timestamp'], unit='ms')
            df['time_played'] = df['time_played']/60000
        except: 
            None
        return df
    else:
        # If there are no new matches to process, print a message and return an empty DataFrame
        print(f'No matches')
        return pd.DataFrame()
    


list_puuid_test = ['jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A']
df_games = api_get_match_history_puuid(list_puuid_test)

def from_df_to_db(df : pd.DataFrame, table = None):
    '''
    Function that convert a df indexed to 0 right index according to db
    & put it 
    '''
    df.reset_index(drop=True, inplace = True)
    df.reset_index(drop=False,inplace = True) 

    starting_index = supabase.table('game_player').select('index').order("index", desc=True).execute().data[0]['index'] #get the latest index in db
    
    # Update the "index" column
    df['index'] = range(starting_index, starting_index + len(df))

    def epoch_to_iso(epoch_time):
        return datetime.utcfromtimestamp(epoch_time / 1000).isoformat() + 'Z'

    for i in range(len(df)-1):
        json_row = df.iloc[i].to_json()
        json_row = json.loads(json_row)
        print(json_row)
        try:
            transformed_data = {
            **json_row,  # Start with the base JSON
            'game_creation': epoch_to_iso(json_row['game_creation']),
            'game_start_timestamp': epoch_to_iso(json_row['game_start_timestamp']),
            'game_end_timestamp': epoch_to_iso(json_row['game_end_timestamp']),
            }
            # Attempt to insert data into the database
            supabase.table(table).insert(transformed_data).execute()

        except Exception as e:
            # Log the exception and row index
            print(f"Failed to insert row {i}. Reason: {str(e)}")
            # Optionally, print the stack trace for debugging
            traceback.print_exc()


from_df_to_db(df_games,'game_player')


jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A
Nombre de matchs avec doublons: 10
Nombre de matchs sans doublons: 10
           match_id                                       participants  \
0   EUW1_7256399856  [JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj...   
0   EUW1_7256399856  [JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj...   
0   EUW1_7256399856  [JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj...   
0   EUW1_7256399856  [JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj...   
0   EUW1_7256399856  [JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj...   
..              ...                                                ...   
0   EUW1_7246792048  [EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptO...   
0   EUW1_7246792048  [EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptO...   
0   EUW1_7246792048  [EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptO...   
0   EUW1_7246792048  [EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptO...   
0   EUW1_7246792048  [EyPINlJgeTr7okD

2025-01-08 00:21:11,775:INFO - HTTP Request: GET https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player?select=index&order=index.desc "HTTP/1.1 200 OK"
C:\Users\anton\AppData\Local\Temp\ipykernel_9108\1869992539.py:365: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(epoch_time / 1000).isoformat() + 'Z'
2025-01-08 00:21:11,822:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 409 Conflict"
Traceback (most recent call last):
  File "C:\Users\anton\AppData\Local\Temp\ipykernel_9108\1869992539.py", line 379, in from_df_to_db
    supabase.table(table).insert(transformed_data).execute()
  File "c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\postgrest\_sync\request_builder.py", line 78, in execute

{'index': 2100, 'match_id': 'EUW1_7256399856', 'participants': ['JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj26NQAJ8gCt9lDeZEHXeeaTlnYj5-1bODA', 'yMjsTrGDImW4Gwf-5Cto3XqVmqeKq750-JwNBfgjtklj0pG1T1FuGbcTNL-JpEk9sSbHH9Hq6YGxvQ', '4npSE6uJ71Oa0lVfjgRXcld5Th0wUW3NgKJW3P594u6ynEnApckUUgga6kHsM5Ab4kM7HjdArmJw4A', 'lgLZavjrqf0f0oPlfESTmdga8dXIiI70uadq5uXQ9DYf_n7YZp-ygC5ydscNGJTREuN3ii2mnJFdyw', 'cO1xaXl2q61jSBq7wQSAZqwogjEq40Ep_s4Knx7o0JELvtHn-HnaLFrurTDYRKPVAksrP1hbk6Dydg', 'RpgoArgWmiiQF93tYL0-WultkkOgZGspQBcpTcPy-JuN98eDgsxQEUX-Ol9Zu24QtOTtIqmA4rcQkA', 'jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A', 'FoejgA27VN1XrXI5X5DutKt67nR1oXe7XC2ivTPLSlOVaW8neTlv9U9YAdcDabgRPX3pzklMwDXCJg', 'FnJduUQ92MN7KJG1hF3SmiwOsA_fLyUSUYirv-mZnY8G6tjIEepVkrIsi2YPGlwFje51tNFhB8d44Q', 'bJ4AxGA5GDk6pfRPM-XtUpy3w2pbjafytZeQEyn69lZpz4Fm4cjGfyQyBJwNqLSA4P5tSDFsffyxqA'], 'game_creation': 1736105626120, 'game_start_timestamp': 1736105641111, 'game_end_timestamp': 1736107391200, 'game_v

2025-01-08 00:21:12,021:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:12,068:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:12,100:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:12,143:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 2105, 'match_id': 'EUW1_7256399856', 'participants': ['JiEKc5VOr9punehCvxMNfS8mIz8dgFB-G-6pWWLX0WYHj26NQAJ8gCt9lDeZEHXeeaTlnYj5-1bODA', 'yMjsTrGDImW4Gwf-5Cto3XqVmqeKq750-JwNBfgjtklj0pG1T1FuGbcTNL-JpEk9sSbHH9Hq6YGxvQ', '4npSE6uJ71Oa0lVfjgRXcld5Th0wUW3NgKJW3P594u6ynEnApckUUgga6kHsM5Ab4kM7HjdArmJw4A', 'lgLZavjrqf0f0oPlfESTmdga8dXIiI70uadq5uXQ9DYf_n7YZp-ygC5ydscNGJTREuN3ii2mnJFdyw', 'cO1xaXl2q61jSBq7wQSAZqwogjEq40Ep_s4Knx7o0JELvtHn-HnaLFrurTDYRKPVAksrP1hbk6Dydg', 'RpgoArgWmiiQF93tYL0-WultkkOgZGspQBcpTcPy-JuN98eDgsxQEUX-Ol9Zu24QtOTtIqmA4rcQkA', 'jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A', 'FoejgA27VN1XrXI5X5DutKt67nR1oXe7XC2ivTPLSlOVaW8neTlv9U9YAdcDabgRPX3pzklMwDXCJg', 'FnJduUQ92MN7KJG1hF3SmiwOsA_fLyUSUYirv-mZnY8G6tjIEepVkrIsi2YPGlwFje51tNFhB8d44Q', 'bJ4AxGA5GDk6pfRPM-XtUpy3w2pbjafytZeQEyn69lZpz4Fm4cjGfyQyBJwNqLSA4P5tSDFsffyxqA'], 'game_creation': 1736105626120, 'game_start_timestamp': 1736105641111, 'game_end_timestamp': 1736107391200, 'game_v

...

{'index': 2184, 'match_id': 'EUW1_7236764126', 'participants': ['DERH9-eANtnHWwwg82tlwHGYYZBwHcAe9BISpBUPw8ZFtsoQPz-FziZQQbrViV_fQk8AJKYqFvibJA', 'vKtGIi3keY6-A4IjL1u_lORggRbFp53tC92G_dLAZ-O4xuwjhtLW09W5BE5cSsqwZo7gUpwrIMy-Vg', 'QD7HC-8Cb7zUsBVTC4AGATEwveHg24kUY-Jht41tlr7PkYLldNKPmflrAmqCS8QcOPIjC6iU8rqmyg', 'U_p3MwGrzpdNH_rGwChMlOI8MXj7_asbL2OaTyOyphvbG4KgndzIeOG2nDK8dJnAa4iwk05y2Y75tw', 'x5iId6Uzd3TaeNsjwwBq87HAOu7IUGqhMsQI-Nu7jkxjbyfMLS4qR829sGgP0xZ_5epDmM1lSMFo_g', '8XRioG577bSUEWiO4OZZKUtQyoyFa8Tt65UAiBCkZJ6kVN2dH70myOCPhiP_sbQxAzkyoMPb05sykg', 'jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A', 'b44dS0vU9Q1uBi8ibwm9bbLBy04Qo4vMRmTRl8ky6C6ac_IhmwfOu-qB27mGZ22-JQGpRf31TvMtFQ', 'V90QKHVsLKhMuueD1svH6RxPJesS1CUTqtmZJx1K4srqrHx_FCzT6iA1rc7jGAjtRwlYnSSo7zsDkA', '8ApEe9CrnPBveGjZ8oHQrdDEmjZPauQOkOa091R8hAMS0lQIZLIC6H9u9AdND-ZzpMm5ySz8UBJNlQ'], 'game_creation': 1734790776796, 'game_start_timestamp': 1734790795573, 'game_end_timestamp': 1734791921733, 'game_v

2025-01-08 00:21:14,998:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,043:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,076:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,112:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,145:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,175:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 2190, 'match_id': 'EUW1_7246792048', 'participants': ['EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptOddmzTICnRb1bWNOtQk60FLA6uiVVY0sOg', 'jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A', 'dsdtTPN3Z_rmXHoIoVqFRS8bLHn9jGKwm_KgR9-lh3hVcngpNOdbcrJL9DlHL-BjzNGg7D68_9UyHA', '3sddVfvZW-x6LLSd7yv4fs2tV1nDZLRz5etS80_coi267JD_kITANqGNOMtGICCadFkZqQnifpDD2Q', 'wQormsydm84vtEzfVM-AnpkzuCtCzQCb45mgIOmB344YT_WrXoydSlJC81KAAJnfWiuzA0OL7FYyyg', 'bSnXI0d0ln-dACfmJ5oMox68rVuupL5nL_epEztH0sFDuHmbmFvGt_44GL0MU37gLF8fe1a9VK-LYA', 'DAP1k0agQoKn0uy9-ujVaaGns43HTsLhJHr6Q5IqJD4c0fx_nY6cUk3uYsFnAvQlHDpAoH4x6rMExg', 'IKHM3VJqRyQLCxQnOzw3xipaii2FOBWqamGuhSuLfLyLWcbSG5MMj7FWIYJPPRBKSSB1JnI-lhFg_g', 'ANkM-d2vXnmYRE8n1ILDpje1rjWLnMdMwJBAcgm7uYw4CNwNQ72MItiTnsKVH3JT2sILaKEuSV2JDw', '4auv5kqlsOMKnpgg77m7JSvUvjhKD27xg6XDVqr_ohlKRILczYRTURzI7RfPhExQ1ZSOHoTCkKjmQg'], 'game_creation': 1735473542520, 'game_start_timestamp': 1735473571974, 'game_end_timestamp': 1735475192657, 'game_v

2025-01-08 00:21:15,207:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,241:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:21:15,274:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 2196, 'match_id': 'EUW1_7246792048', 'participants': ['EyPINlJgeTr7okDrfrmqY0l0SzCY-eI6h-yb9zzgdLptOddmzTICnRb1bWNOtQk60FLA6uiVVY0sOg', 'jNWsZnNbVN4b9yl2PclSu9ZFELcEPStRlggmlNc_y7jccg61MVdONmilckULYsFzwgOfVwseUmpp1A', 'dsdtTPN3Z_rmXHoIoVqFRS8bLHn9jGKwm_KgR9-lh3hVcngpNOdbcrJL9DlHL-BjzNGg7D68_9UyHA', '3sddVfvZW-x6LLSd7yv4fs2tV1nDZLRz5etS80_coi267JD_kITANqGNOMtGICCadFkZqQnifpDD2Q', 'wQormsydm84vtEzfVM-AnpkzuCtCzQCb45mgIOmB344YT_WrXoydSlJC81KAAJnfWiuzA0OL7FYyyg', 'bSnXI0d0ln-dACfmJ5oMox68rVuupL5nL_epEztH0sFDuHmbmFvGt_44GL0MU37gLF8fe1a9VK-LYA', 'DAP1k0agQoKn0uy9-ujVaaGns43HTsLhJHr6Q5IqJD4c0fx_nY6cUk3uYsFnAvQlHDpAoH4x6rMExg', 'IKHM3VJqRyQLCxQnOzw3xipaii2FOBWqamGuhSuLfLyLWcbSG5MMj7FWIYJPPRBKSSB1JnI-lhFg_g', 'ANkM-d2vXnmYRE8n1ILDpje1rjWLnMdMwJBAcgm7uYw4CNwNQ72MItiTnsKVH3JT2sILaKEuSV2JDw', '4auv5kqlsOMKnpgg77m7JSvUvjhKD27xg6XDVqr_ohlKRILczYRTURzI7RfPhExQ1ZSOHoTCkKjmQg'], 'game_creation': 1735473542520, 'game_start_timestamp': 1735473571974, 'game_end_timestamp': 1735475192657, 'game_v

2025-01-08 00:20:58,768:INFO - HTTP Request: GET https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player?select=index&order=index.desc "HTTP/1.1 200 OK"


2100

MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw
Nombre de matchs avec doublons: 82
Nombre de matchs sans doublons: 82


In [115]:


df_temp = df.copy()
from_df_to_db(df_temp,'game_player')

2025-01-08 00:07:35,494:INFO - HTTP Request: GET https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player?select=%2A "HTTP/1.1 200 OK"
C:\Users\anton\AppData\Local\Temp\ipykernel_9108\989441024.py:18: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(epoch_time / 1000).isoformat() + 'Z'
2025-01-08 00:07:35,708:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,745:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,780:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,820:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase

{'index': 1901, 'match_id': 'EUW1_7057789467', 'participants': ['SCgF2SpndmINjJ4ImcR6hmryvxMj_hEcK7fYpK_WBJ6SIBLSawPEQ692azr3KZHPbqAYkAHIKDKGvg', 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw'], 'game_creation': 1722603087617, 'game_start_timestamp': 1722603097979, 'game_end_timestamp': 1722603490719, 'game_version': '14.15.604.8769', 'queue_id': 1820, 'game_mode': 'STRAWBERRY', 'platform_id': 'EUW1', 'puuid': 'SCgF2SpndmINjJ4ImcR6hmryvxMj_hEcK7fYpK_WBJ6SIBLSawPEQ692azr3KZHPbqAYkAHIKDKGvg', 'riot_id': 'Irilindies', 'riot_tag': 'EUW', 'time_played': 6.5456666667, 'side': 'blue', 'win': False, 'team_position': '', 'lane': 'NONE', 'champion': 'Strawberry_Jinx', 'kills': 0, 'deaths': 1, 'assists': 0, 'summoner1_id': 6, 'summoner2_id': 7, 'gold_earned': 205, 'total_minions_killed': 474, 'total_neutral_minions_killed': 0, 'total_ally_jungle_minions_killed': 0, 'total_enemy_jungle_minions_killed': 0, 'early_surrender': False, 'surrender': False, 'first_blood'

2025-01-08 00:07:35,854:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,912:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,949:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:35,990:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,025:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,057:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 1906, 'match_id': 'EUW1_7062605489', 'participants': ['MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw'], 'game_creation': 1722799609340, 'game_start_timestamp': 1722799617764, 'game_end_timestamp': 1722799953074, 'game_version': '14.15.604.8769', 'queue_id': 1810, 'game_mode': 'STRAWBERRY', 'platform_id': 'EUW1', 'puuid': 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw', 'riot_id': 'cocow', 'riot_tag': 'EUW', 'time_played': 5.5885, 'side': 'blue', 'win': False, 'team_position': '', 'lane': 'NONE', 'champion': 'Strawberry_Jinx', 'kills': 0, 'deaths': 1, 'assists': 0, 'summoner1_id': 6, 'summoner2_id': 7, 'gold_earned': 405, 'total_minions_killed': 772, 'total_neutral_minions_killed': 0, 'total_ally_jungle_minions_killed': 0, 'total_enemy_jungle_minions_killed': 0, 'early_surrender': False, 'surrender': False, 'first_blood': False, 'first_blood_assist': False, 'first_tower': False, 'first_tower_assist': False, 'dam

2025-01-08 00:07:36,102:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,142:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,181:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,233:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:36,266:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


...

{'index': 2085, 'match_id': 'EUW1_7062134622', 'participants': ['gZ52_y8RGX7eHBpX_pk9iu86ibEEN7TDjJy2KGXXBOL8p9uhjrgtteRKBYUFj20sKDag4oxS1ObXTg', 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw', 'lS2MRx0uzb7VEfoTe1rLOLBtNVKvUcgTUf1yEkBXNnddXLScY6fqEd_qUYrTTsTJ5eLHpluv8sT6kA'], 'game_creation': 1722787747323, 'game_start_timestamp': 1722787767132, 'game_end_timestamp': 1722789171967, 'game_version': '14.15.604.8769', 'queue_id': 1830, 'game_mode': 'STRAWBERRY', 'platform_id': 'EUW1', 'puuid': 'lS2MRx0uzb7VEfoTe1rLOLBtNVKvUcgTUf1yEkBXNnddXLScY6fqEd_qUYrTTsTJ5eLHpluv8sT6kA', 'riot_id': 'Suiper Troll', 'riot_tag': 'g0g0I', 'time_played': 23.4139166667, 'side': 'blue', 'win': False, 'team_position': '', 'lane': 'NONE', 'champion': 'Strawberry_Xayah', 'kills': 0, 'deaths': 2, 'assists': 0, 'summoner1_id': 6, 'summoner2_id': 7, 'gold_earned': 5775, 'total_minions_killed': 7253, 'total_neutral_minions_killed': 0, 'total_ally_jungle_minions_killed': 0, 'total_en

2025-01-08 00:07:42,631:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,666:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,706:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,736:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,768:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,802:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 2090, 'match_id': 'EUW1_7057402145', 'participants': ['SCgF2SpndmINjJ4ImcR6hmryvxMj_hEcK7fYpK_WBJ6SIBLSawPEQ692azr3KZHPbqAYkAHIKDKGvg', 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw'], 'game_creation': 1722584087005, 'game_start_timestamp': 1722584102892, 'game_end_timestamp': 1722584608322, 'game_version': '14.15.604.8769', 'queue_id': 1820, 'game_mode': 'STRAWBERRY', 'platform_id': 'EUW1', 'puuid': 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw', 'riot_id': 'cocow', 'riot_tag': 'EUW', 'time_played': 8.4238333333, 'side': 'blue', 'win': False, 'team_position': '', 'lane': 'NONE', 'champion': 'Strawberry_Jinx', 'kills': 0, 'deaths': 2, 'assists': 0, 'summoner1_id': 6, 'summoner2_id': 7, 'gold_earned': 288, 'total_minions_killed': 1483, 'total_neutral_minions_killed': 0, 'total_ally_jungle_minions_killed': 0, 'total_enemy_jungle_minions_killed': 0, 'early_surrender': False, 'surrender': False, 'first_blood': Fa

2025-01-08 00:07:42,842:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,870:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,902:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,936:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"
2025-01-08 00:07:42,967:INFO - HTTP Request: POST https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player "HTTP/1.1 201 Created"


{'index': 2096, 'match_id': 'EUW1_7061948062', 'participants': ['MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw', 'lS2MRx0uzb7VEfoTe1rLOLBtNVKvUcgTUf1yEkBXNnddXLScY6fqEd_qUYrTTsTJ5eLHpluv8sT6kA'], 'game_creation': 1722782985619, 'game_start_timestamp': 1722782993871, 'game_end_timestamp': 1722783419597, 'game_version': '14.15.604.8769', 'queue_id': 1820, 'game_mode': 'STRAWBERRY', 'platform_id': 'EUW1', 'puuid': 'MqegYubZUQBWNJpd6Ydq7CvjydOqGJFPbkssXA1u0rmAbs5IQ7XI0gUqVSEGcK2IHFu89IsMxkv5Qw', 'riot_id': 'cocow', 'riot_tag': 'EUW', 'time_played': 7.0954333333, 'side': 'blue', 'win': False, 'team_position': '', 'lane': 'NONE', 'champion': 'Strawberry_Riven', 'kills': 0, 'deaths': 4, 'assists': 0, 'summoner1_id': 6, 'summoner2_id': 7, 'gold_earned': 615, 'total_minions_killed': 366, 'total_neutral_minions_killed': 0, 'total_ally_jungle_minions_killed': 0, 'total_enemy_jungle_minions_killed': 0, 'early_surrender': False, 'surrender': False, 'first_blood': Fa

In [125]:
len(supabase.table('game_player').select('*').execute().data)



2025-01-08 00:00:43,921:INFO - HTTP Request: GET https://ykxyohbbokadclxisvch.supabase.co/rest/v1/game_player?select=%2A "HTTP/1.1 200 OK"


1901